In [ ]:
import os
import skimage.io as skio
import matplotlib.pyplot as plt
from pathlib import Path

def inspect_weights(
    weights_directory: str,
    images_paths: list,
    results_folder: str = "/Users/yahyarahhawi/Developer/Film/inspected_weights/results",
    resize_to: int = 2048,
    device: str = "mps"
):
    """
    Recursively finds all '.pth' files in 'weights_directory', and for each file
    applies transform_image_A_to_B() to all images in 'images_paths'. Saves results
    under 'results_folder', grouping by each weights file, but combining subfolders
    with underscores. E.g., 'cinestill/100_net_G_A.pth' => 'cinestill_100_net_G_A.pth'

    Additionally, for each weights file, it compiles a single Matplotlib figure
    showing all transformed images in a 3×4 layout (up to 12 images), and saves
    it as "<folder_name>.png" in the `results_folder`.
    """
    # Ensure results folder exists
    os.makedirs(results_folder, exist_ok=True)

    for root, dirs, files in os.walk(weights_directory):
        for file_name in files:
            if file_name.endswith(".pth"):
                weights_path = os.path.join(root, file_name)
                relative_root = os.path.relpath(root, weights_directory)

                # Convert the subfolder path + filename into one underscore-based name
                # e.g. "cinestill" + "100_net_G_A.pth" => "cinestill_100_net_G_A.pth"
                # If 'relative_root' is '.', it means it's the top-level folder
                if relative_root == ".":
                    folder_name = file_name  # e.g., "100_net_G_A.pth"
                else:
                    folder_name = relative_root + "_" + file_name  # e.g., "cinestill_100_net_G_A.pth"

                # Replace path separators with underscores (on Windows/Mac/Linux)
                folder_name = folder_name.replace(os.sep, "_")

                # Construct final output folder for individual images
                results_subdir = os.path.join(results_folder, folder_name)
                os.makedirs(results_subdir, exist_ok=True)

                print(f"\n=== Processing weights: {weights_path}")
                print(f"=== Output folder for images: {results_subdir}")

                # We'll keep track of the (filename, image) pairs to compile the subplot later
                output_images = []

                for img_path in images_paths:
                    try:
                        out_img = transform_image_A_to_B(
                            input_image=img_path,
                            weights_path=weights_path,
                            device=device,
                            resize_to=resize_to
                        )
                        img_filename = os.path.basename(img_path)
                        out_path = os.path.join(results_subdir, img_filename)
                        skio.imsave(out_path, out_img)
                        print(f"    Saved: {out_path}")

                        # Add to our list for plotting
                        output_images.append((img_filename, out_img))

                    except Exception as e:
                        print(f"    Error processing {img_path} with {weights_path}: {e}")

                # ---------------------------------------------------------
                # Create a single figure with subplots (3 rows x 4 columns)
                # ---------------------------------------------------------
                if output_images:
                    # We'll only show up to 12 images in the grid
                    max_images = min(len(output_images), 12)

                    fig, axes = plt.subplots(3, 4, figsize=(16, 12))
                    axes = axes.flatten()  # Flatten 3x4 to a list of 12 axes

                    for i in range(max_images):
                        img_filename, out_img = output_images[i]
                        ax = axes[i]
                        ax.imshow(out_img)
                        ax.set_title(img_filename, fontsize=8)
                        ax.axis("off")

                    # If there are fewer than 12 images, hide the remaining axes
                    for j in range(max_images, 12):
                        axes[j].axis("off")

                    plt.tight_layout()

                    # Save the compiled figure
                    # e.g. if folder_name="cinestill_100_net_G_A.pth", 
                    # compiled image = "cinestill_100_net_G_A.pth.png" in results_folder
                    compiled_plot_path = os.path.join(
                        results_folder, folder_name + ".png"
                    )
                    plt.savefig(compiled_plot_path, dpi=150)
                    plt.close(fig)

                    print(f"=== Compiled figure saved: {compiled_plot_path}")
                else:
                    print("No images were processed; no compiled figure created.")

In [ ]:
images_paths = [ "/Users/yahyarahhawi/Developer/Film/pytorch-CycleGAN-and-pix2pix/datasets/film_transfer/trainA/_nusrattt_1692914685_3176721087780991322_8028270133.jpg"
                , "/Users/yahyarahhawi/Developer/Film/pytorch-CycleGAN-and-pix2pix/datasets/film_transfer/trainA/9sglr0n2tlae1.jpeg"
                , "/Users/yahyarahhawi/Developer/Film/pytorch-CycleGAN-and-pix2pix/datasets/film_transfer/trainA/iphone-15-pm-v0-vbmtquw0lx9e1.jpeg"
                , "/Users/yahyarahhawi/Developer/Film/pytorch-CycleGAN-and-pix2pix/datasets/film_transfer/trainA/434148921_1215316256115590_8824329964826277346_n.jpg"
                , "/Users/yahyarahhawi/Developer/Film/pytorch-CycleGAN-and-pix2pix/datasets/film_transfer/trainA/468098174_10160452228782283_6470774328062649638_n.jpg"
                , "/Users/yahyarahhawi/Developer/Film/pytorch-CycleGAN-and-pix2pix/datasets/film_transfer/trainA/468367136_1223915785349049_272835853960648938_n.jpg"
                , "/Users/yahyarahhawi/Developer/Film/pytorch-CycleGAN-and-pix2pix/datasets/film_transfer/trainA/471580547_10235710514767354_6503898022093232902_n.jpg"
                , "/Users/yahyarahhawi/Developer/Film/pytorch-CycleGAN-and-pix2pix/datasets/film_transfer/trainA/472209162_609603644783753_187943241519109706_n.jpg"
                , "/Users/yahyarahhawi/Developer/Film/pytorch-CycleGAN-and-pix2pix/datasets/film_transfer/trainA/472521894_10160974665802895_1246614851933189592_n.jpg"
                , "/Users/yahyarahhawi/Developer/Film/pytorch-CycleGAN-and-pix2pix/datasets/film_transfer/trainA/472573341_10222111737781000_47137509779831210_n.jpg"
                , "/Users/yahyarahhawi/Developer/Film/pytorch-CycleGAN-and-pix2pix/datasets/film_transfer/trainA/amsal.pt2_1692328736_3171805793028838612_59580996889.jpg"
                , "/Users/yahyarahhawi/Developer/Film/pytorch-CycleGAN-and-pix2pix/datasets/film_transfer/trainA/autumn-in-austria-iphone-15-pro-v0-hw13g12e3iud1.jpeg"]

In [ ]:
inspect_weights(
    weights_directory="/Users/yahyarahhawi/Developer/Film/weights",
    images_paths=images_paths,
    results_folder="/Users/yahyarahhawi/Developer/Film/inspected_weights/results",
    resize_to=2048,
    device="mps"
)